In [92]:
import sys, os
sys.path.append(os.path.abspath('..'))
import numpy as np
from ImageLab import *
import cv2
from PIL import Image

The two points in each cluster that have the max distance between them are the points that define the extreme lines in the curve.

In [93]:

# Assuming the points are given as a list of tuples
def left_right(points):
  # Initialize the leftmost and rightmost points
  left = points[0]
  right = points[0]
  # Loop through all points
  for point in points:
    # Compare the x-coordinates of the points
    if point[0] < left[0]:
      # Update the leftmost point
      left = point
    elif point[0] > right[0]:
      # Update the rightmost point
      right = point
  # Return the leftmost and rightmost points
  return (left, right)

# Example
points = [(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)]
print(left_right(points))
# Output: ((1, 2), (9, 10))

((1, 2), (9, 10))


Clustering Algorithm

In [94]:
# Import the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

# Define a function that takes a list of points and returns the labels and the number of clusters
def dbscan_cluster(points):
  # Convert the points to a numpy array
  points = np.squeeze(np.array(points))
  # Create a dbscan object with some parameters
  dbscan = DBSCAN(eps=0.5, min_samples=5)
  # Fit the dbscan object to the points and get the labels
  labels = dbscan.fit_predict(points)
  # Create an empty dictionary to store the lists of points
  clusters = {}

  # Iterate over the unique labels
  for label in np.unique(labels):

      # Filter the points that belong to the current label
      points = points[labels == label]

      # Convert the points to a list of tuples
      points = [tuple(p) for p in points]

      # Store the list of points in the dictionary with the label as the key
      clusters[label] = points
  return clusters


# The output of this code is:

# {-1: [(25, 80)], 0: [(1, 2), (2, 2), (2, 3)], 1: [(8, 7), (8, 8)]}
# As you can see, the dictionary contains three keys: -1 for the outlier, 0 and 1 for the two clusters. The values are the lists of points that belong to each cluster.

This function applies the necessary hough operations on the segmented image.

In [95]:
import numpy as np
import math

def hough_lines(image, threshold, folder, name):
  # image is a binary array with 0s indicating non-edges and 1s indicating edges
  # returns a list of tuples (rho, theta) representing lines in polar space
  
  # get the dimensions of the image
  height, width = image.shape
  
  # get the maximum possible distance from the origin to any point in the image
  max_dist = int(math.sqrt(height**2 + width**2))
  
  # create an array of theta values from -90 to 90 degrees
  theta = np.deg2rad(np.arange(-90, 90))
  
  # create an empty accumulator array with dimensions (rho, theta)
  accumulator = np.zeros((2 * max_dist, len(theta)))

  # loop through each pixel in the image
  for x in range(width):
    for y in range(height):
      # if the pixel is an edge point
      if image[y, x] > 0:
        # calculate the rho values for each theta value
        rhos = x * np.cos(theta) + y * np.sin(theta)
        # round the rho values to the nearest integer
        rhos = np.round(rhos).astype(int)
        for i in range(len(rhos)):
          # increment the corresponding cell in the accumulator array
          accumulator[int(rhos[i] + max_dist), int(np.rad2deg(theta[i])) + 90] += 1
  
  
  stretched_houghs = ImageFilters(accumulator).process(contrast_stretch())
  
  _=ImageUtil(stretched_houghs).save_image_to_folder('accumulator', name)
    
  segmented_houghs = ImageSegment(stretched_houghs).process(Global_Threshold(threshold))
  segmented = segmented_houghs * stretched_houghs
  
  _=ImageUtil(segmented).save_image_to_folder('segmented', name)
  
  return segmented

Function Pipeline for detection

In [96]:
kernel = np.array([[0, 1, 0],
                  [1, 1, 1],
                  [0, 1, 0]])


def pipeline(img_path, folder, name):
    
    image = Image.open(img_path).convert('L')
    image = np.array(image)

    dilated = ImageProcessor(image).process(Dilation(), kernel)
    closed = ImageProcessor(dilated).process(Erosion(), kernel)
    sauvola = ImageSegment(closed).process(Pixel_Filter(3, 50, 'Sauvola'))

    seggrey = sauvola * closed
    seggrey = np.squeeze(seggrey)
    
    hough_accumulator = hough_lines(seggrey, 120, folder, name)
    
    labels, clusters = dbscan_cluster(hough_accumulator)

    return seggrey

In [97]:
# set the path to your test images folder
target_folder = 'test_images'
output_folder = 'detection_images'

chi_difference = 1

# loop through each image file in the folder
for file_name in os.listdir(target_folder):
    # check if the file is an image file
    if file_name.endswith('.jpg') or file_name.endswith('.png'):
        
        img_path = os.path.join(target_folder, file_name)
        file_name = os.path.splitext(file_name)[0]
        
        segimg = pipeline(img_path, 'Segmented', f'{file_name}.png')
                